In [1]:
import sys

import matplotlib.pyplot as plt
import json

from thom import *
from thom.base import *
from thom.utils import *

from private.lyap import lyap_r, lyap_e, corr_dim

%load_ext autoreload
%autoreload 2

In [2]:
try:
    from private.lyap import lyap_r, lyap_e, corr_dim
except:
    from nolds import lyap_r, lyap_e, corr_dim

In [3]:
# We will make a local copy of the internal database
OUTPUT_FILE = "./chaotic_attractors2.json"
INPUT_FILE = "thom/data/chaotic_attractors.json"
RECALCULATE = True

total_length = 40000*100
total_length = 500000

with open(INPUT_FILE, "r") as file:
    data = json.load(file)
print(len(data.keys()))

for i, item in enumerate(list(data.keys())):
    
    model = getattr(thom, item)()
    tpts, sol = model.make_trajectory(10000, resample=True, pts_per_period=300, return_times=True)
    dt = np.median(np.diff(tpts))
    print(item, ": ", end="")
    
    if "maximum_lyapunov_estimated" not in data[item] or RECALCULATE:
        lyap = lyap_r(sol.T, tau=dt)
        data[item]["maximum_lyapunov_estimated"] = lyap
        print(f"lyap: {lyap} ", end="")
    
    if "correlation_dimension" not in data[item] or RECALCULATE:
        cdim = corr_dim(sol.T)
        data[item]["correlation_dimension"] = cdim
        print(f"corr_dim: {cdim} ", end="")
    
#     if "permutation_entropy" not in data[item] or RECALCULATE:
        
        
    print("\n")
    # Save new file
    with open(OUTPUT_FILE, 'w') as f:
        json.dump(data, f, indent=4)

#     eq = getattr(thom, item)()
#     sol = eq.make_trajectory(total_length, method="Radau")
    
# #     all_freqs = list()
# #     for comp in sol:
# #         try:
# #             freqs, amps = find_significant_frequencies(comp, return_amplitudes=True, significance_threshold=0.95)
# #             max_freq = freqs[np.argmax(np.abs(amps))]
# #         except:
# #             ## ignores time-like coordinates
# #             continue
# #         all_freqs.append(max_freq)
# #         print(".", end='')
# #     period = eq.dt * (1 / np.median(all_freqs))
    
# #     freqs, amps = find_significant_frequencies(sol[0], return_amplitudes=True, significance_threshold=0.95)
# #     max_freq = freqs[np.argmax(np.abs(amps))]
# #     period = (1/max_freq) * eq.dt
    
#     #period = np.median([1/freq_from_fft(item) * eq.dt for item in sol])
    
#     data[item]["period"] = signif(period, 5)
#     print(item, " time period", period," index period", period/eq.dt, " ", (1/eq.dt)/period)
    


113
Lorenz : 

/Users/williamgilpin/program_repos/thom/private/lyap.py:332: RuntimeWarning: signal has very low mean frequency, setting min_tsep = 2500
  warnings.warn(msg.format(min_tsep), RuntimeWarning)


lyap: 0.9200316147079158 corr_dim: 1.9611111444516849 

LorenzBounded : 

/Users/williamgilpin/program_repos/thom/private/lyap.py:332: RuntimeWarning: signal has very low mean frequency, setting min_tsep = 2500
  warnings.warn(msg.format(min_tsep), RuntimeWarning)


lyap: 0.6444812234320899 corr_dim: 1.9494161061592994 

LorenzCoupled : 

...

lyap: 0.11025370726734905 corr_dim: 1.6823665904621319 

SprottTorus : 

/Users/williamgilpin/program_repos/thom/private/lyap.py:332: RuntimeWarning: signal has very low mean frequency, setting min_tsep = 2500
  warnings.warn(msg.format(min_tsep), RuntimeWarning)


lyap: 0.1173788893597375 corr_dim: 1.006920639170857 

SprottJerk : 

/Users/williamgilpin/program_repos/thom/private/lyap.py:332: RuntimeWarning: signal has very low mean frequency, setting min_tsep = 2500
  warnings.warn(msg.format(min_tsep), RuntimeWarning)


KeyboardInterrupt: 

In [12]:
print("t", end="")
print("i")

ti


In [11]:

for i in range(4):
    if True:
        pass
        print("y")

    print("t")

y
t
y
t
y
t
y
t


In [4]:
hasattr(data["Lorenz"], "period")

False

In [6]:
"period" in data["Lorenz"]

True